# HW2

- tasks
    - 划分训练集和验证集；
    - 按照 MMPreTrain CustomDataset 格式组织训练集和验证集；
    - 使用 MMPreTrain 算法库，编写配置文件，正确加载预训练模型；
    - 在水果数据集上进行微调训练；
    - 使用 MMPreTrain 的 ImageClassificationInferencer 接口，对网络水果图像，或自己拍摄的水果图像，使用训练好的模型进行分类。
- 需提交的验证集评估指标（不能低于 60%）

![example acc image](https://user-images.githubusercontent.com/66227028/243670989-d72eb206-9df3-44f6-8ad5-28ae70858fa2.png)

## 1. split train and val dataset


In [12]:
import os
import numpy

import shutil
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

PROJECT_ROOT = '/home/louzekun/projects/mmlab/mmpretrain'

os.chdir(PROJECT_ROOT)
os.getcwd()


'/mnt/data/ai4phys/louzekun/projects/mmlab/mmpretrain'

In [2]:
raw_dpath = '/home/louzekun/projects/mmlab/mmpretrain/data/fruit30_train'
dataset_dpath = '/home/louzekun/projects/mmlab/mmpretrain/data/fruit30_dataset'  # to be created
train_dpath = '/home/louzekun/projects/mmlab/mmpretrain/data/fruit30_dataset/train'
val_dpath = '/home/louzekun/projects/mmlab/mmpretrain/data/fruit30_dataset/val'

# make dataset dir
os.makedirs(dataset_dpath, mode=0o776, exist_ok=True)
os.makedirs(train_dpath, mode=0o776, exist_ok=True)
os.makedirs(val_dpath, mode=0o776, exist_ok=True)


In [3]:
# rename raw_dpath subdirs to English names
print(os.listdir(raw_dpath))
# translate with newbing
raw_sub_dnames = ['脐橙', '西瓜', '香蕉', '柠檬', '梨', '桂圆', '草莓', '胡萝卜', '石榴', '西红柿', '苦瓜', '菠萝', '猕猴桃', '黄瓜', '圣女果', '葡萄-红', '椰子', '车厘子', '葡萄-白', '榴莲', '柚子', '火龙果', '杨梅', '苹果-青', '砂糖橘', '苹果-红', '哈密瓜', '荔枝', '山竹', '芒果']
# newbing prompt is:
# ranslate these names into English, use short names as possible, output in python List[str] format, replace blanck with underline:
trans_sub_dnames = ['navel_orange', 'watermelon', 'banana', 'lemon', 'pear', 'longan', 'strawberry', 'carrot', 'pomegranate', 'tomato', 'bitter_melon', 'pineapple', 'kiwi_fruit', 'cucumber', 'cherry_tomato', 'grape_red', 'coconut', 'cherry', 'grape_white', 'durian', 'pomelo', 'dragon_fruit', 'waxberry', 'apple_green', 'tangerine_orange', 'apple_red', 'hami_melon', 'lychee', 'mangosteen', 'mango']
len(raw_sub_dnames), len(trans_sub_dnames)

['dragon_fruit', 'lemon', 'longan', 'navel_orange', 'cherry', 'tangerine_orange', 'cucumber', 'pineapple', 'grape_red', 'pomelo', 'hami_melon', 'durian', 'kiwi_fruit', 'tomato', 'bitter_melon', 'waxberry', 'grape_white', 'cherry_tomato', 'watermelon', 'pomegranate', 'banana', 'mango', 'apple_red', 'carrot', 'strawberry', 'lychee', 'pear', 'apple_green', 'coconut', 'blip-base_3rdparty_coco-caption_20230419-a5b71af3.pth', 'mangosteen']


(30, 30)

In [5]:
# rename subdirs to English names
for chname, enname in zip(raw_sub_dnames, trans_sub_dnames):
    chdpath = os.path.join(raw_dpath, chname)
    endpath = os.path.join(raw_dpath, enname)
    print(chname, '->', enname)
    shutil.move(chdpath, endpath)


In [6]:
train_frac, val_frac = 0.8, 0.2
# normalize
train_frac, val_frac = train_frac / (train_frac + val_frac), val_frac / (train_frac + val_frac)
raw_en_sub_dnames = [d for d in os.listdir(raw_dpath) if os.path.isdir(os.path.join(raw_dpath, d))]

for raw_sub_dname in raw_en_sub_dnames:
    raw_sub_dpath = os.path.join(raw_dpath, raw_sub_dname)
    train_sub_dpath = os.path.join(train_dpath, raw_sub_dname)
    val_sub_dpath = os.path.join(val_dpath, raw_sub_dname)
    _cp = lambda dfrom, dto, fname: shutil.copy(os.path.join(dfrom, fname), os.path.join(dto, fname))
    for d in [train_sub_dpath, val_sub_dpath]:
        if os.path.exists(d): shutil.rmtree(d)
        os.makedirs(d, mode=0o776, exist_ok=True)

    sub_fnames = os.listdir(raw_sub_dpath)
    # shuffle
    numpy.random.shuffle(sub_fnames)
    # split
    train_cnt = int(len(sub_fnames) * train_frac)
    train_fnames = sub_fnames[:train_cnt]
    val_fnames = sub_fnames[train_cnt:]
    print(f"dirname: {raw_sub_dname:16s}\tfiles: {len(sub_fnames)}\ttrain: {len(train_fnames)}\tval: {len(val_fnames)}")
    # copy to train and val dirs
    for fname in tqdm(train_fnames, ncols=100, desc=f"copying {raw_sub_dname} to train"):
        _cp(raw_sub_dpath, train_sub_dpath, fname)
    for fname in tqdm(val_fnames, ncols=100, desc=f"copying {raw_sub_dname} to val"):
        _cp(raw_sub_dpath, val_sub_dpath, fname)

dirname: dragon_fruit    	files: 146	train: 116	val: 30


copying dragon_fruit to val: 100%|██████████████████████████████████| 30/30 [00:00<00:00, 69.37it/s]


dirname: lemon           	files: 119	train: 95	val: 24


copying lemon to val: 100%|█████████████████████████████████████████| 24/24 [00:00<00:00, 69.15it/s]


dirname: longan          	files: 153	train: 122	val: 31


...

copying apple_green to val: 100%|███████████████████████████████████| 31/31 [00:00<00:00, 78.18it/s]


dirname: coconut         	files: 154	train: 123	val: 31


copying coconut to val: 100%|███████████████████████████████████████| 31/31 [00:00<00:00, 71.26it/s]


dirname: mangosteen      	files: 143	train: 114	val: 29


copying mangosteen to val: 100%|████████████████████████████████████| 29/29 [00:00<00:00, 73.23it/s]


In [ ]:
# check train and val dirs files cnt
# skip this step


In [13]:
# load all the images and check for RGB mean and std

dataset_dpath = '/home/louzekun/projects/mmlab/mmpretrain/data/fruit30_dataset'  # to be created
# find all the images path recursively
img_fpaths = []
for root, dirs, files in os.walk(dataset_dpath):
    for fname in files:
        if fname.endswith('.jpg'):
            img_fpaths.append(os.path.join(root, fname))

len(img_fpaths)

# load each image and convert to shape (RGB, pixels)
# then compute the mean and std
img_data = []
for img_fpath in tqdm(img_fpaths, ncols=100, desc="loading images"):
    img = Image.open(img_fpath)
    img_data.append(numpy.array(img))


loading images: 100%|██████████████████████████████████████████| 3715/3715 [00:13<00:00, 269.78it/s]


In [15]:
img_data = numpy.concatenate([d.reshape(-1, 3) for d in img_data], axis=0)
img_data.shape

(967589988, 3)

In [20]:
import torch

device = torch.device('cuda:0')
img_data = torch.from_numpy(img_data).to(device)

In [22]:
# convert from torch.uint8 to torch.float32
img_data = img_data.float()

In [24]:
img_data.mean(dim=0), img_data.std(dim=0)

(tensor([169.1138, 147.0236, 113.2977], device='cuda:0'),
 tensor([77.2956, 79.5745, 89.1477], device='cuda:0'))

## 2. organize dataset to MMPreTrain CustomDataset format


In [ ]:
# already done
